In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
from google.colab import drive
import os
from os import listdir
import librosa
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
from tqdm import tqdm
import pandas as pd
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

#만약 GPU를 사용 가능하다면 device 값이 cuda가 되고, 아니라면 cpu가 됩니다.

device='cpu'
# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정


    #학습에 사용할 파라미터를 설정합니다.

learning_rate = 0.001
training_epochs = 15
batch_size = 100
#데이터로더를 사용하여 데이터를 다루기 위해서 데이터셋을 정의해줍니다.

data1 = np.loadtxt("/content/drive/MyDrive/Colab Notebooks/train/Rock_TRAIN.txt")
data2 = np.loadtxt("/content/drive/MyDrive/Colab Notebooks/test/Rock_TEST.txt")
"""
mnist_train = dsets.MNIST(root='Rock data/Rock_TEST.arff', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='Rock_data/Rock_TRAIN.arff', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)
#데이터로더를 사용하여 배치 크기를 지정해줍니다. 만약 데이터셋과 데이터로더가 기억이 안 난다면 '미니 배치와 데이터 로드' 챕터를 꼭 복습하세요.
"""
data_loader = torch.utils.data.DataLoader(dataset=data1,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)
#이제 클래스로 모델을 설계합니다.

class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv1d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out
#모델을 정의합니다.

# CNN 모델 정의
model = CNN().to(device)
#비용 함수와 옵티마이저를 정의합니다.

criterion = torch.nn.MSELoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#총 배치의 수를 출력해보겠습니다.

total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))
#총 배치의 수 : 600
#총 배치의 수는 600입니다. 그런데 배치 크기를 100으로 했으므로 결국 훈련 데이터는 총 60,000개란 의미입니다. 이제 모델을 훈련시켜보겠습니다. (시간이 꽤 오래 걸립니다.)

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    #print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))


#이제 테스트를 해보겠습니다.

# 학습을 진행하지 않을 것이므로 torch.no_grad()

with torch.no_grad():
    X_test = data2.test_data.view(len(data2), 1, 2844).float().to(device)
    Y_test = data2.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())


총 배치의 수 : 0


AttributeError: ignored